In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# handle gpu leakage issue on DSI cluster
# ideally we shouldn't have to do this but leaving it for now in case the issue pops up again
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [19]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

import scripts

In [20]:
# Confirm GPUs are working properly and set default device explicitly
device = "cuda:0"
torch.cuda.device_count()

1

In [4]:
MODEL_DIR = "/net/projects/veitch/LLMs/"
MODEL = "llama1-based-models/alpaca-7b" # or llama2-based-models/llama2-hf/Llama-2-7b-chat-hf
MODEL_PATH = MODEL_DIR + MODEL

In [5]:
# TODO: Clean this up

# This cell is for doing ROME edits
# ROME model and tokenizer
# MODEL_NAME = "gpt2-xl"
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.pad_token = tokenizer.eos_token

In [6]:
tokenizer = transformers.LlamaTokenizer.from_pretrained(MODEL_PATH)
model = transformers.LlamaForCausalLM.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto").to(device)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [30]:
LABELS = "title" # choices are "title" and "categories"
DATA_FOLDER = "data/"
TRAIN_DATA = DATA_FOLDER + "train-12-articles.csv"
VAL_DATA = DATA_FOLDER + "val-12-articles.csv"
PROBE_TYPE = "linear" # choices are "linear" and "mlp"
LAYER = -1
EPOCHS = 200
BATCH_SIZE = 4
AGGREGATION = "max" # choices are "max" or "mean"
SMOKE_TEST = False # TODO: this doesn't work yet
PRINT_PROGRESS = False

In [31]:
scripts.train_handler(model, tokenizer, device, TRAIN_DATA, VAL_DATA, probe_type=PROBE_TYPE, labels=LABELS, layer=LAYER, aggregation=AGGREGATION, batch_size=BATCH_SIZE, epochs=EPOCHS, smoke_test=SMOKE_TEST, print_progress=PRINT_PROGRESS)

[Training][5] loss: 1.871
[Validation][5] loss: 0.216
[Validation]5 accuracy: 0.806
[Training][10] loss: 1.324
[Validation][10] loss: 0.127
[Validation]10 accuracy: 0.861
[Training][15] loss: 0.216
[Validation][15] loss: 0.144
[Validation]15 accuracy: 0.833
[Training][20] loss: 0.000
[Validation][20] loss: 0.189
[Validation]20 accuracy: 0.889
[Training][25] loss: 0.000
[Validation][25] loss: 0.191
[Validation]25 accuracy: 0.889
[Training][30] loss: 0.000
[Validation][30] loss: 0.178
[Validation]30 accuracy: 0.917
[Training][35] loss: 0.000
[Validation][35] loss: 0.164
[Validation]35 accuracy: 0.917
[Training][40] loss: 0.000
[Validation][40] loss: 0.148
[Validation]40 accuracy: 0.917
[Training][45] loss: 0.000
[Validation][45] loss: 0.133
[Validation]45 accuracy: 0.917
[Training][50] loss: 0.000
[Validation][50] loss: 0.117
[Validation]50 accuracy: 0.917
[Training][55] loss: 0.000
[Validation][55] loss: 0.100
[Validation]55 accuracy: 0.917
[Training][60] loss: 0.000
[Validation][60] lo

Linear(
  (fc1): Linear(in_features=4096, out_features=12, bias=True)
)